(C) Logan Vasudea, 2020 

# Logan's DIY Covid-19 Dashboard

This Covid dashboard will use Python and the data published by Public Health England to analyse how Covid-19 has affected the UK.


In [ ]:
from uk_covid19 import Cov19API
import datetime
import json
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wdg
#from ipywidgets import Layout, Button, Box, GridspecLayout
from IPython.display import Markdown, display, HTML

# an iPython  "magic" that enables the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 150

def printmd(string):
    """Prints in markdown format"""
    display(Markdown(string))

In [ ]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

In [ ]:
def column_rename(df,rename_dict):
    """Renames columns in a data frame when given a dictionary for renaming"""
    df.rename(columns=rename_dict, inplace=True)

In [ ]:
# Loads json file for date vs no. of cumulative cases, admissions, deaths, and no. of new cases, admissions, deaths
# Above data will be stored in df1
with open("date_case_adm_death.json", "rt") as INFILE:
    rawdata1=json.load(INFILE)
 


In [ ]:
#the dictionaries for renaming the columns in df1 

def wrangle_data1(data):
    """ Returns a dataframe from json data for:
    date vs no. of cumulative cases, admissions, deaths, and no. of new cases, admissions, deaths. """
    legend_to_struct_dictionary1={
        "Number of cases":"cases",
        "Number of admissions":"admissions",
        "Number of deaths":"deaths",
        "Date": "date",
        "Number of new cases":"noOfNewCases",
        "Number of new Hospital Admissions":"noOfNewHospitalAdmissions",
        "Number of new deaths":"noOfNewDeaths"
    }
    struct_to_legend_dictionary1=dict(list(zip(legend_to_struct_dictionary1.values(), legend_to_struct_dictionary1.keys())))


    datalist=data["data"]
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    columns_list=['noOfNewCases','cases', 'noOfNewHospitalAdmissions','admissions', 'noOfNewDeaths', 'deaths' ]
    date_case_adm_death_df=pd.DataFrame(index=index, columns=columns_list)
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in columns_list:
            #if pd.isna(date_case_adm_death_df.loc[date, column]): 
            #excluded above line as decided to update all data each time as data may change
            #None values wil be NaN in the dataframe as this data has not been made available by PHE. 
            #It is likely that they will be updated later for these dates
                if entry[column]!=None: 
                    value= float(entry[column])
                    date_case_adm_death_df.loc[date, column]=value
    
    column_rename(date_case_adm_death_df,struct_to_legend_dictionary1)
    return date_case_adm_death_df



In [ ]:
df1=wrangle_data1(rawdata1) #creates dataframe for rawdata11

In [ ]:
#The filters and structure for the API call for the first graph

filters1 = [
    'areaType=overview' # note each metric-value pair is inside one string
]

structure1 = {
    "date": "date",
    "noOfNewCases": "newCasesByPublishDate",
    "cases": "cumCasesByPublishDate",
    "noOfNewHospitalAdmissions": "newAdmissions",
    "admissions": "cumAdmissions",
    "noOfNewDeaths": "newDeaths28DaysByDeathDate",
    "deaths": "cumDeaths28DaysByDeathDate"    
}

In [ ]:
output1 = wdg.Output() #creates widget output object

@output1.capture(clear_output=True,wait=True)
def access_api1(button):
    """Accessing API for graph1 and updating dataframe """
    filters1 = [
        'areaType=overview' # note each metric-value pair is inside one string
    ]
    structure1 = {
        "date": "date",
        "noOfNewCases": "newCasesByPublishDate",
        "cases": "cumCasesByPublishDate",
        "noOfNewHospitalAdmissions": "newAdmissions",
        "admissions": "cumAdmissions",
        "noOfNewDeaths": "newDeaths28DaysByDeathDate",
        "deaths": "cumDeaths28DaysByDeathDate"    
    }
    
    api1 = Cov19API(filters=filters1, structure=structure1)
    printmd("I'm downloading data from the API...")
    try:
        rawdata1 = api1.get_json()
        printmd("...all done.")
        now=datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S").split()
        printmd("Data refreshed on " + now[0] + " at " + now[1])
    except:
        printmd("Unable to update data")
        
    try: 
        with open("date_case_adm_death.json", "wt") as OUTFILE:
            json.dump(rawdata1, OUTFILE)
    except:
        printmd("Unable to update json")
    global df1
    df1=wrangle_data1(rawdata1)
    refresh_graph1()
    refresh_graph2()


    
    
#the button        
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api1)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)
output1

In [ ]:
for col in df1.columns:
    df1[col].values[:] = 1

## Cumulative Totals of Cases, Admissions and Deaths

Below is a graph showing the cumulative totals of cases, hospital admissions and deaths in the UK. As of late November 2020, there have been over a million cases with 200,000 of those admitted to hospital. Sadly, there have been over 60,000 deaths.

In [ ]:
def time_series_graph1(lines):
    """Function to plot graph 1"""
    n_lines=len(lines)
    
    if n_lines>0:
        graph_data_frame=df1[list(lines)]
        ax=graph_data_frame.plot()
        ax.set_title("Cumulative Totals of Important Statistics")
        #ax.ticklabel_format(axis='y', style='plain')
       

    else:
        printmd("Select the data you want to print.")
        
    

In [ ]:
def num_of_cumulative_cases_text(obs):
    """ Function to explain the drop in cases on the 2nd of July """
    tup=obs
    text=""
    for item in tup:
        if item=='Number of cases':
            text="Number of cases refers to the individuals who have had at least one positive COVID-19 test result (either lab-reported or lateral flow device), by date reported. On 2 July, case data from Pillars 1 and 2 of the testing programme were combined and de-duplicated, resulting in a step decrease in the cumulative number of cases reported."
    printmd(text)

In [ ]:
lines_to_print1=wdg.SelectMultiple(
    options=["Number of cases", "Number of admissions", "Number of deaths"],
    value=["Number of cases", "Number of admissions", "Number of deaths"],
    rows=3,
    description='Stats:',
    disabled=False
    
)

graph1=wdg.interactive_output(time_series_graph1, {'lines':lines_to_print1})
text_to_print1=wdg.interactive_output(num_of_cumulative_cases_text, {'obs':lines_to_print1})

In [ ]:
grid1=wdg.GridspecLayout(5,2)
grid1[:,0]=graph1
grid1[2:,1] = text_to_print1
grid1[1,1] = lines_to_print1

In [ ]:
display(grid1)

In [ ]:
def refresh_graph1():
    """ We change the value of the widget in order to force a redraw of the graph."""
    current=lines_to_print1.value
    if current==lines_to_print1.options[:]:
        other=[]
        lines_to_print1.value=other
        lines_to_print1.value=current
    else:
        other=lines_to_print1.options[:]
        lines_to_print1.value=other


## Daily Totals of New Cases, Admissions and Deaths

This graph show the number of new cases, hospital admissions or deaths on each day (for the whole of the UK). The UK is currently (as of November) in the middle of a second wave, though thankfully the number of deaths in this second wave is lower. Please note: The dates for the data are the actual date the even occurs, so the data may change as the PHE publish more data.

In [ ]:
lines_to_print2=wdg.SelectMultiple(
    options=["Number of new cases","Number of new Hospital Admissions", "Number of new deaths"],
    value=["Number of new cases","Number of new Hospital Admissions", "Number of new deaths"],
    rows=3,
    description='Stats:',
    disabled=False
    
)

In [ ]:
def time_series_graph2(lines):
    """Function to plot graph 2"""
    n_lines=len(lines)
    if n_lines>0:
        graph_data_frame=df1[list(lines)]
        ax=graph_data_frame.plot()
        ax.set_title("Daily Totals of Important Statistics")
        #ax.ticklabel_format(axis='y', style='plain')
    else:
        printmd("Select the data you want to print.")
        

In [ ]:
graph2=wdg.interactive_output(time_series_graph2, {'lines':lines_to_print2})
grid2=wdg.GridspecLayout(5,2)
grid2[:,0]=graph2
#grid[2:,1] = text_to_print1
grid2[1,1] = lines_to_print2

display(grid2)

In [ ]:
def refresh_graph2():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=lines_to_print2.value
    if current==lines_to_print2.options[:]:
        other=[]
        lines_to_print2.value=other
        lines_to_print2.value=current
    else:
        other=lines_to_print2.options[:]
        lines_to_print2.value=other